# Instalação de Pacotes com `pip`

Este comando instala vários pacotes Python necessários para desenvolver modelos de Machine Learning e manipular dados. Vamos detalhar cada parte:



In [4]:
!pip install azureml-sdk pandas scikit-learn matplotlib

# Detecção de Anomalias com Isolation Forest

Este código realiza detecção de anomalias em um conjunto de dados numéricos utilizando o algoritmo **Isolation Forest**. Ele inclui as etapas de carregamento, pré-processamento e treinamento do modelo, além de salvar o modelo treinado para uso futuro. Abaixo, cada etapa é explicada em detalhes:

## Bibliotecas Utilizadas

- **pandas**: Utilizado para carregar e manipular o conjunto de dados.
- **sklearn.preprocessing.StandardScaler**: Utilizado para escalar os dados numéricos, garantindo que todas as features tenham a mesma escala.
- **sklearn.ensemble.IsolationForest**: Algoritmo de detecção de anomalias utilizado para identificar dados fora do padrão.
- **joblib**: Biblioteca usada para salvar o modelo treinado em um arquivo, permitindo reutilizá-lo sem precisar treiná-lo novamente.

## Etapa 1: Carregamento dos Dados

```python
file_path = 'order_status_code.csv'
data = pd.read_csv(file_path)
```

- O código carrega um arquivo CSV chamado `order_status_code.csv` contendo os dados que serão usados para detecção de anomalias.

## Etapa 2: Pré-processamento

- Primeiro, o código verifica os tipos de dados das colunas para identificar se há colunas de tipo `datetime`.

```python
print("Data types:")
print(data.dtypes)
```

- Se houver uma coluna de data/hora chamada `datetime_column`, ela é convertida para o formato `datetime`. Em seguida, recursos úteis, como **hora**, **dia**, **mês** e **ano**, são extraídos e a coluna original é removida.

```python
if 'datetime_column' in data.columns:
    data['datetime_column'] = pd.to_datetime(data['datetime_column'])
    data['hour'] = data['datetime_column'].dt.hour
    data['day'] = data['datetime_column'].dt.day
    data['month'] = data['datetime_column'].dt.month
    data['year'] = data['datetime_column'].dt.year
    data = data.drop(columns=['datetime_column'])
```

## Etapa 3: Seleção de Colunas Numéricas

```python
data_numeric = data.select_dtypes(include=[float, int])
data_numeric = data_numeric.dropna().drop_duplicates()
```

- Apenas colunas numéricas são selecionadas, já que o algoritmo **Isolation Forest** trabalha apenas com dados numéricos.
- Linhas com valores ausentes ou duplicados são removidas para garantir a qualidade dos dados.

## Etapa 4: Escalonamento dos Dados

```python
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_numeric)
```

- Os dados numéricos são escalonados usando `StandardScaler` para padronizar as variáveis, ou seja, ajustá-las para terem média zero e desvio padrão 1. Isso melhora o desempenho do modelo.

## Etapa 5: Treinamento do Modelo de Detecção de Anomalias

```python
model = IsolationForest(contamination=0.01)
model.fit(X_scaled)
```

- O modelo **Isolation Forest** é instanciado e treinado usando os dados escalados.
- O parâmetro `contamination=0.01` indica que o modelo deve considerar 1% dos dados como anômalos.

## Etapa 6: Salvamento do Modelo Treinado

```python
joblib.dump(model, 'anomaly_detection_model.pkl')
```

- O modelo treinado é salvo em um arquivo `.pkl` usando a biblioteca `joblib`. Isso permite que o modelo seja carregado e utilizado posteriormente sem precisar ser treinado novamente.

## Conclusão

Este código realiza a detecção de anomalias em um conjunto de dados numéricos usando o **Isolation Forest**. Ele inclui todas as etapas essenciais para carregar os dados, pré-processá-los, treinar o modelo e, finalmente, salvar o modelo para uso futuro. O modelo pode ser usado em um ambiente de produção para detectar anomalias em novos dados.


In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import joblib

# Step 1: Load your data
file_path = 'order_status_code.csv'
data = pd.read_csv(file_path)

# Step 2: Preprocessing (Handle datetime columns, select only numerical columns, etc.)
# Check if there are any datetime columns that need to be converted
print("Data types:")
print(data.dtypes)

# If you have a datetime column, convert it and extract useful features (like hour, day, etc.)
# Example: Assuming there's a column named 'datetime_column'
if 'datetime_column' in data.columns:
    data['datetime_column'] = pd.to_datetime(data['datetime_column'])
    data['hour'] = data['datetime_column'].dt.hour
    data['day'] = data['datetime_column'].dt.day
    data['month'] = data['datetime_column'].dt.month
    data['year'] = data['datetime_column'].dt.year
    # Drop the original datetime column
    data = data.drop(columns=['datetime_column'])

# Step 3: Select only numeric columns for scaling and training
data_numeric = data.select_dtypes(include=[float, int])

# Remove rows with missing values and duplicates
data_numeric = data_numeric.dropna().drop_duplicates()

# Step 4: Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_numeric)

# Step 5: Train the Isolation Forest model for anomaly detection
# Adjust the contamination as needed (1% anomalies)
model = IsolationForest(contamination=0.01) 
model.fit(X_scaled)

# Step 6: Save the trained model for deployment
joblib.dump(model, 'anomaly_detection_model.pkl')

print("Model training complete and saved as 'anomaly_detection_model.pkl'")


Data types:
code    int64
dtype: object
Model training complete and saved as 'anomaly_detection_model.pkl'


# Conectando ao Workspace do Azure Machine Learning

Este código configura o ambiente do Azure Machine Learning e estabelece uma conexão com o workspace. Aqui está uma explicação detalhada de cada parte:

## 1. **Importando a Classe Workspace**

```python
from azureml.core import Workspace
```

- O **Workspace** é um contêiner central no Azure Machine Learning que contém todos os seus ativos de machine learning, como experimentos, modelos, pipelines e computação. 
- A função `Workspace` vem do SDK do **AzureML** e é utilizada para conectar-se ao ambiente de trabalho da sua conta no Azure.

## 2. **Conectando ao Workspace do Azure ML**

```python
ws = Workspace.from_config()
```

- Este comando tenta carregar a configuração do **Workspace** a partir de um arquivo `config.json`, que contém os detalhes de autenticação e identificação do seu workspace (como o nome do recurso, assinatura do Azure e grupo de recursos).
- Se o arquivo `config.json` estiver corretamente configurado no diretório do projeto, essa linha estabelecerá a conexão automaticamente.
- Caso contrário, seria necessário passar as informações manualmente (por exemplo, nome do workspace, ID de assinatura do Azure e nome do grupo de recursos).

## 3. **Exibindo Detalhes do Workspace**

```python
ws.get_details()
```

- Esta linha exibe informações sobre o workspace ao qual você está conectado, como:
  - Nome do workspace
  - Localização
  - ID de assinatura do Azure
  - Grupo de recursos associado
  - Outras propriedades do workspace

## Resumo

Este código estabelece uma conexão com o **Azure Machine Learning Workspace** usando o SDK do Azure ML. Ele facilita o gerenciamento de recursos e experimentos em um ambiente de machine learning na nuvem. A função `from_config()` simplifica o processo, permitindo que as credenciais sejam carregadas automaticamente de um arquivo de configuração. Uma vez conectado, você pode começar a realizar operações de machine learning, como criação de experimentos, gerenciamento de modelos e implantação.

In [3]:
# Step 1: Set up the Azure environment
from azureml.core import Workspace

# Connect to Azure ML workspace (use config.json or manually input workspace details)
ws = Workspace.from_config()

# Display workspace details
ws.get_details()


{'id': '/subscriptions/32855dba-b7b2-4fdb-8288-f0615afa0cdf/resourceGroups/rg-evaluations/providers/Microsoft.MachineLearningServices/workspaces/ml-workspace-training',
 'name': 'ml-workspace-training',
 'identity': {'principal_id': '739ef407-6415-40b3-8bc1-6ba9fafc63bf',
  'tenant_id': '6bd72601-6bad-49d9-82df-a390b167cd68',
  'type': 'SystemAssigned'},
 'location': 'swedencentral',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '2e136fb4-42a8-4e2f-8d62-51567db61c8f',
 'sdkTelemetryAppInsightsKey': 'ecfc0110-6630-4d5b-935f-f8d304673557',
 'description': '',
 'friendlyName': 'Machine Learning Training Workspace',
 'creationTime': '2024-10-14T17:02:16.9671419Z',
 'containerRegistry': '/subscriptions/32855dba-b7b2-4fdb-8288-f0615afa0cdf/resourceGroups/rg-evaluations/providers/Microsoft.ContainerRegistry/registries/2e136fb442a84e2f8d6251567db61c8f',
 'keyVault': '/subscriptions/32855dba-b7b2-4fdb-8288-f0615afa0cdf/resourceGroups/rg-e

# Registrando um Modelo Treinado no Azure Machine Learning

Este código faz o registro de um modelo de machine learning no **Azure Machine Learning Workspace**. A seguir, está a explicação de cada parte:

## 1. **Importando a Classe `Model`**

```python
from azureml.core.model import Model
```

- A classe `Model` do **AzureML SDK** é usada para gerenciar modelos no workspace. Ela permite registrar, listar e baixar modelos para experimentos e deployment.
- O registro de um modelo no Azure facilita a rastreabilidade e reutilização do modelo para futuras implementações ou avaliações.

## 2. **Registrando o Modelo no Workspace**

```python
model = Model.register(
    workspace=ws, 
    model_path="anomaly_detection_model.pkl",  # Caminho para o modelo treinado
    model_name="anomaly_detection_model"  # Nome do modelo registrado no Azure
)
```

- A função `Model.register()` realiza o registro do modelo no **workspace** do Azure.
- **Parâmetros**:
  - `workspace`: refere-se ao workspace do Azure Machine Learning em que o modelo será registrado. Esse workspace já foi criado e conectado na etapa anterior (`ws`).
  - `model_path`: o caminho local do arquivo do modelo treinado (neste caso, `anomaly_detection_model.pkl`), que será carregado e registrado no Azure.
  - `model_name`: o nome com o qual o modelo será registrado no Azure Machine Learning, neste exemplo, `"anomaly_detection_model"`. Esse nome será usado para identificar e acessar o modelo posteriormente.

## 3. **Verificando o Registro do Modelo**

```python
print(f"Model registered with ID: {model.id}")
```

- Depois que o modelo é registrado com sucesso, o código exibe o **ID do modelo** no workspace. Esse ID pode ser utilizado para referenciar o modelo em futuras operações, como ao fazer deploy ou monitorar o desempenho do modelo.

## Resumo

Este código registra um modelo treinado no **Azure Machine Learning**. O registro do modelo facilita sua gestão e reutilização, permitindo que seja facilmente implantado em novos ambientes ou compartilhado com outros membros da equipe. A função `Model.register()` carrega o arquivo do modelo do caminho especificado e o associa ao workspace no Azure, atribuindo um nome e gerando um ID único para rastrear o modelo. Esse processo é essencial em um fluxo de machine learning, especialmente em ambientes de produção e colaborativos.

In [4]:
# Step 2: Register the trained model in Azure
from azureml.core.model import Model

# Register the model in the workspace
model = Model.register(workspace=ws,
                       model_path="anomaly_detection_model.pkl",  # Path to the trained model
                       model_name="anomaly_detection_model")  # Name of the model in Azure

# Print the model ID to verify registration
print(f"Model registered with ID: {model.id}")


Registering model anomaly_detection_model
Model registered with ID: anomaly_detection_model:4


# Criando um Ambiente para Inferência de Modelos no Azure Machine Learning

Este código cria e configura um ambiente personalizado no **Azure Machine Learning** para ser usado durante a inferência de um modelo de machine learning. Abaixo está uma explicação detalhada de cada parte do código.

## 1. **Importando a Classe `Environment`**

```python
from azureml.core import Environment
```

- A classe `Environment` do **AzureML SDK** é usada para definir o ambiente em que os modelos serão treinados ou executados. Um ambiente contém as bibliotecas e dependências necessárias para que o código seja executado corretamente no Azure, seja em um processo de treinamento ou durante a inferência.

## 2. **Criando o Ambiente para Inferência**

```python
env = Environment(name="anomaly-env")
```

- Aqui, um novo ambiente chamado `"anomaly-env"` é criado. Esse ambiente será utilizado para hospedar o modelo de detecção de anomalias e garantir que todas as dependências necessárias estejam presentes para realizar a inferência de dados.
- Um **ambiente** é uma forma de garantir que o código do modelo tenha todas as bibliotecas e versões corretas disponíveis quando for executado em um contêiner no Azure.

## 3. **Adicionando Dependências ao Ambiente**

```python
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("azureml-inference-server-http")
```

- Esta parte do código adiciona as bibliotecas necessárias ao ambiente.
  - **`scikit-learn`**: É a biblioteca que contém o algoritmo de **Isolation Forest** e outras ferramentas de machine learning.
  - **`joblib`**: Usada para salvar e carregar o modelo treinado. A `joblib` permite serializar e desserializar o modelo de forma eficiente.
  - **`numpy`**: Biblioteca fundamental para cálculos numéricos e manipulação de arrays, amplamente usada em machine learning.
  - **`azureml-inference-server-http`**: Este pacote é essencial para hospedar o modelo no Azure e permitir que ele seja acessado via uma API HTTP para realizar inferências (predições) sobre novos dados.

## Resumo

Este código configura um **ambiente** personalizado no **Azure Machine Learning** para a execução de inferências de um modelo de detecção de anomalias. Ele cria o ambiente chamado `"anomaly-env"` e adiciona todas as dependências necessárias (bibliotecas Python) para que o modelo seja executado corretamente no ambiente de produção, permitindo que o modelo faça predições sobre novos dados. O uso de ambientes no Azure ML garante que o código seja executado de forma consistente e previsível em diferentes fases, seja no desenvolvimento, teste ou produção.

In [8]:
from azureml.core import Environment

# Criar o ambiente para a inferência do modelo
env = Environment(name="anomaly-env")

# Adicionar pacotes necessários (como scikit-learn, joblib, numpy e o servidor HTTP)
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("joblib")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("azureml-inference-server-http")


# Configuração de Inferência no Azure Machine Learning

Este código define a configuração necessária para implantar um modelo de machine learning no **Azure Machine Learning** com base no script de inferência e no ambiente criado anteriormente. Abaixo está a explicação detalhada de cada parte:

## 1. **Importando a Classe `InferenceConfig`**

```python
from azureml.core.model import InferenceConfig
```

- A classe `InferenceConfig` do **AzureML SDK** é usada para configurar como o modelo será implantado e executado para inferência. Ela define o script de entrada e o ambiente onde o modelo será carregado e usado para realizar predições.

## 2. **Definindo a Configuração de Inferência**

```python
inference_config = InferenceConfig(entry_script="score.py", environment=env)
```

- Aqui, é criada uma instância da classe `InferenceConfig`, que especifica as informações necessárias para a execução do modelo na fase de inferência.

### Parâmetros:
- **`entry_script="score.py"`**:
  - O script de entrada que será usado para executar o modelo. O arquivo `score.py` geralmente contém a lógica de carregamento do modelo, o processamento de entrada e a geração de predições.
  - Dentro do script `score.py`, funções como `init()` (para inicializar o modelo) e `run()` (para realizar inferências sobre os dados de entrada) são definidas.
  
- **`environment=env`**:
  - O ambiente que foi criado anteriormente no código, chamado `"anomaly-env"`, contendo todas as dependências necessárias (bibliotecas como `scikit-learn`, `joblib`, `numpy` e o servidor HTTP do Azure).
  - Isso garante que o modelo terá acesso às mesmas bibliotecas e versões que foram usadas durante o treinamento ou preparação para a inferência.

## Resumo

Este código define a **configuração de inferência** para implantar o modelo no **Azure Machine Learning**. Ele associa o script de inferência, `score.py`, que contém a lógica para carregar e executar o modelo, ao ambiente previamente configurado (`env`). O arquivo `score.py` é fundamental, pois ele determina como o modelo processa dados de entrada e produz predições, enquanto o ambiente garante que todas as dependências necessárias estejam disponíveis durante a execução do modelo.

Essa configuração é crucial para a implantação de modelos no Azure, permitindo que o modelo seja acessado e utilizado por meio de uma API ou interface de inferência para previsões em tempo real.

In [2]:
from azureml.core.model import InferenceConfig

# Definir a configuração de inferência usando o arquivo score.py
inference_config = InferenceConfig(entry_script="score.py", environment=env)


# Simulando `init()` e `run()` para Inferência de Anomalias com Azure ML

Este código simula as funções principais que são geralmente usadas em um script de inferência no **Azure Machine Learning**, como o arquivo `score.py`. Ele inclui a inicialização e execução do modelo para realizar predições com base em dados de entrada e é estruturado para detectar anomalias.

## 1. **Importando Bibliotecas Necessárias**

```python
import json
import numpy as np
import joblib
import os
import random
```

- **`json`**: Usado para carregar e processar os dados de entrada que estão no formato JSON.
- **`numpy`**: Usado para manipular os dados de entrada como arrays NumPy, que são a estrutura esperada para a maioria dos algoritmos de machine learning.
- **`joblib`**: Biblioteca usada para carregar o modelo previamente salvo em um arquivo `.pkl`.
- **`os`**: Biblioteca para interagir com o sistema de arquivos, como verificar se o modelo existe.
- **`random`**: Utilizada para gerar dados de teste com uma distribuição específica para simular erros HTTP.

## 2. **Função `init()`**

```python
def init():
    global model
    model_path = 'anomaly_detection_model.pkl'
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Modelo não encontrado no caminho: {model_path}")
    
    model = joblib.load(model_path)
```

- A função `init()` é responsável por inicializar o modelo. Em um ambiente de produção no Azure, o modelo seria carregado do diretório `AZUREML_MODEL_DIR`, mas aqui é usado o caminho local `'anomaly_detection_model.pkl'`.
- Se o modelo não for encontrado no caminho especificado, é gerado um erro `FileNotFoundError`.
- O modelo é carregado usando o **`joblib.load()`**, que desserializa o modelo para estar pronto para realizar inferências.

## 3. **Função `run()`**

```python
def run(raw_data):
    try:
        data = json.loads(raw_data)['input']
        input_data = np.array(data)
        
        expected_features = model.n_features_in_
        if input_data.shape[1] != expected_features:
            raise ValueError(f"O modelo espera {expected_features} features, mas os dados de entrada possuem {input_data.shape[1]} features.")
        
        predictions = model.predict(input_data)
        alerts = [f"Anomaly detected at row {i}" for i, pred in enumerate(predictions) if pred == -1]
        
        return {"predictions": predictions.tolist(), "alerts": alerts}
    except Exception as e:
        return {"error": str(e)}
```

- **Entrada**:
  - Recebe os dados de entrada em formato JSON. O JSON deve conter uma chave chamada `'input'` que armazena os dados em formato de matriz (ou lista de listas).
  
- **Processamento**:
  - Converte os dados de entrada de JSON para uma matriz NumPy.
  - Verifica se o número de **features** (colunas) no conjunto de dados de entrada corresponde ao número de features para o qual o modelo foi treinado. Se o número de features não corresponder, um erro é gerado.
  
- **Inferência**:
  - Usa o modelo carregado para fazer previsões. No caso de anomalias, o modelo retorna `-1`, indicando que os dados correspondentes são anômalos.
  
- **Alertas**:
  - Gera uma lista de alertas para cada linha de entrada onde uma anomalia (`-1`) foi detectada.

- **Saída**:
  - Retorna um JSON contendo as **previsões** e os **alertas** gerados.

## 4. **Execução Local para Testes**

```python
if __name__ == "__main__":
    init()

    http_statuses = [500] * 85 + random.choices([403, 500, 400], k=15)
    
    test_input = json.dumps({
        "input": [[status] for status in http_statuses]
    })
    
    result = run(test_input)
    print(result)
```

- **Simulação de Erros HTTP**:
  - Aqui, são gerados códigos de status HTTP, onde 85% das requisições são 500 (indicado como "normais"), e 15% são erros HTTP aleatórios (403, 400 e 500) simulando dados de entrada.
  
- **Testes**:
  - A função `run()` é chamada com esses dados simulados para verificar como o modelo detecta anomalias. O resultado é impresso no console.

## Resumo

Este código simula o processo de inferência para detecção de anomalias com base em um modelo de **Isolation Forest** no Azure Machine Learning. As funções **`init()`** e **`run()`** são típicas de scripts de inferência no Azure, onde o modelo é carregado uma vez e as predições são feitas em dados de entrada recebidos em formato JSON. O código também inclui um teste local para verificar como o modelo lida com dados simulados e detectar anomalias entre os códigos de status HTTP fornecidos.

In [5]:
import json
import numpy as np
import joblib
import os
import random

# Simular a função init() do score.py
def init():
    """
    Esta função inicializa o modelo. Em produção, o modelo seria carregado do diretório
    AZUREML_MODEL_DIR, mas aqui podemos carregar o modelo diretamente do local.
    """
    global model
    model_path = 'anomaly_detection_model.pkl'  # Caminho local para testar
    
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Modelo não encontrado no caminho: {model_path}")
    
    # Carregar o modelo com joblib
    model = joblib.load(model_path)

# Simular a função run() do score.py
def run(raw_data):
    """
    Esta função processa os dados de entrada e realiza a inferência.
    """
    try:
        # Analisar o JSON de entrada
        data = json.loads(raw_data)['input']
        
        # Converter os dados em uma matriz NumPy para realizar a previsão
        input_data = np.array(data)
        
        # Verificar quantas features o modelo foi treinado para receber
        expected_features = model.n_features_in_  # Número de features esperado pelo modelo
        if input_data.shape[1] != expected_features:
            raise ValueError(f"O modelo espera {expected_features} features, mas os dados de entrada possuem {input_data.shape[1]} features.")
        
        # Fazer as previsões usando o modelo carregado
        predictions = model.predict(input_data)
        
        # Gerar alertas para quaisquer anomalias detectadas (previsão = -1)
        alerts = [f"Anomaly detected at row {i}" for i, pred in enumerate(predictions) if pred == -1]
        
        # Retornar as previsões e os alertas como um JSON
        return {"predictions": predictions.tolist(), "alerts": alerts}
    
    except Exception as e:
        return {"error": str(e)}

# Simular a execução local para testar
if __name__ == "__main__":
    init()  # Inicializar o modelo

    # Gerar percentual de erro de 15%
    http_statuses = [200] * 85 + random.choices([403, 500, 400], k=15)
    
    # Exemplo de dados de entrada no formato JSON
    test_input = json.dumps({
        "input": [[status] for status in http_statuses] 
    })
    
    # Executar a função run() com os dados de teste
    result = run(test_input)
    print(result)


{'predictions': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'alerts': []}


# Deletando um Serviço Web no Azure Machine Learning

Este código carrega um serviço web já existente no **Azure Machine Learning** e, em seguida, exclui esse serviço. Abaixo está a explicação de cada parte do código:

## 1. **Importando a Classe `Webservice`**

```python
from azureml.core.webservice import Webservice
```

- A classe `Webservice` do **AzureML SDK** é usada para gerenciar serviços web que foram implantados para hospedar modelos de machine learning no Azure. Esses serviços permitem que o modelo seja acessado por meio de uma API para realizar inferências.

## 2. **Carregando o Serviço Web Existente**

```python
service = Webservice(workspace=ws, name="anomaly-detector-service")
```

- Esta linha carrega um serviço web que já foi implantado anteriormente. 
- **Parâmetros**:
  - **`workspace=ws`**: O workspace do Azure Machine Learning onde o serviço está hospedado. Esse workspace deve estar previamente configurado e conectado.
  - **`name="anomaly-detector-service"`**: O nome do serviço web que você deseja gerenciar ou deletar. Esse nome deve corresponder ao nome do serviço registrado no workspace.

## 3. **Excluindo o Serviço Web**

```python
service.delete()
```

- A função `delete()` é chamada para excluir o serviço web especificado. Isso significa que o modelo não estará mais disponível para ser acessado por uma API, e os recursos associados a ele, como a infraestrutura de contêineres, serão liberados.
- Esta ação é irreversível: uma vez que o serviço é deletado, ele precisará ser reimplantado caso seja necessário novamente.

## 4. **Confirmando a Exclusão**

```python
print("Serviço deletado com sucesso.")
```

- Após a exclusão bem-sucedida, uma mensagem é exibida no console confirmando que o serviço foi deletado.

## Resumo

Este código carrega um serviço web já implantado no **Azure Machine Learning** e o exclui. O serviço web é responsável por hospedar um modelo de machine learning, permitindo que ele seja acessado via API para inferências. A função `delete()` remove o serviço e libera os recursos associados, interrompendo o acesso ao modelo. A exclusão de um serviço pode ser útil quando você não precisa mais dele, economizando custos e recursos computacionais no Azure.

In [13]:
from azureml.core.webservice import Webservice

# Carregar o serviço existente
service = Webservice(workspace=ws, name="anomaly-detector-service")

# Excluir o serviço
service.delete()
print("Serviço deletado com sucesso.")

Running
2024-10-14 19:57:10+00:00 Deleting service.
2024-10-14 19:57:15+00:00 Deleting service entity.
Succeeded
Serviço deletado com sucesso.


# Implantando um Modelo no Azure Container Instances (ACI)

Este código implanta um modelo de machine learning no **Azure Container Instances (ACI)**, um serviço gerenciado do Azure que permite implantar contêineres sem precisar gerenciar a infraestrutura subjacente. O modelo será disponibilizado como um serviço web para realizar inferências em tempo real. Abaixo está a explicação de cada parte do código.

## 1. **Importando Bibliotecas Necessárias**

```python
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
```

- **`AciWebservice`**: Classe usada para configurar e implantar um serviço de modelo no **Azure Container Instances (ACI)**.
- **`Model`**: Classe usada para carregar e gerenciar o modelo previamente registrado no workspace do **Azure Machine Learning**.

## 2. **Configurando o Deployment para o ACI**

```python
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
```

- Aqui, uma configuração de implantação para o **ACI** é criada, definindo os recursos de computação a serem alocados para o contêiner que hospedará o modelo.
- **Parâmetros**:
  - **`cpu_cores=1`**: Define a quantidade de CPUs a ser alocada para o contêiner.
  - **`memory_gb=1`**: Define a quantidade de memória (em GB) a ser alocada para o contêiner.
  
Essas configurações são ajustáveis dependendo da complexidade do modelo e da carga esperada.

## 3. **Implantando o Modelo no ACI**

```python
service = Model.deploy(
    workspace=ws,
    name="anomaly-detector-service",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)
```

- **`Model.deploy()`**: Esta função implanta o modelo no **ACI**. Os parâmetros necessários incluem:
  - **`workspace=ws`**: O workspace do Azure Machine Learning onde o modelo está registrado.
  - **`name="anomaly-detector-service"`**: O nome do serviço que será criado e exposto para inferências.
  - **`models=[model]`**: O modelo que foi registrado no workspace e que será implantado. Este modelo já foi carregado anteriormente.
  - **`inference_config=inference_config`**: A configuração de inferência, que define o script de pontuação (`score.py`) e o ambiente com as dependências necessárias (configurado anteriormente).
  - **`deployment_config=aci_config`**: A configuração do ACI, que define os recursos computacionais para o contêiner.

## 4. **Aguardando a Conclusão da Implantação**

```python
service.wait_for_deployment(show_output=True)
```

- Esta função aguarda a conclusão do processo de implantação, exibindo no console as mensagens de log associadas ao processo. Quando a implantação for concluída, o serviço estará disponível para uso.

## 5. **Exibindo a URI de Pontuação**

```python
print(f"Service deployed at: {service.scoring_uri}")
```

- Após a implantação ser concluída, o código exibe a **URI de pontuação** (ou scoring URI), que é a URL na qual o serviço pode ser acessado para realizar inferências via API.
- As requisições de inferência podem ser enviadas para essa URI com dados no formato apropriado (geralmente JSON) para obter as predições do modelo.

## Resumo

Este código faz o deployment de um modelo de machine learning no **Azure Container Instances (ACI)**. O processo inclui a configuração de recursos de computação, a definição de como o modelo será executado para realizar inferências e, finalmente, a implantação do serviço web no ACI. Ao término, o serviço estará acessível por uma API, permitindo que clientes enviem dados para receber predições do modelo.

A implantação no ACI é ideal para testes e pequenos workloads de inferência, uma vez que oferece escalabilidade sem a necessidade de gerenciamento da infraestrutura.

In [14]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

# Configuração para deployment no ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Realizar a implantação do modelo
service = Model.deploy(workspace=ws,
                       name="anomaly-detector-service",
                       models=[model],  # Modelo que já foi registrado anteriormente
                       inference_config=inference_config,
                       deployment_config=aci_config)

# Aguardar até a conclusão da implantação
service.wait_for_deployment(show_output=True)

# Exibir a URI de pontuação para enviar requisições
print(f"Service deployed at: {service.scoring_uri}")


/tmp/ipykernel_3353/1094109139.py:8: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-10-14 19:57:48+00:00 Creating Container Registry if not exists.
2024-10-14 19:57:50+00:00 Building image..
2024-10-14 20:09:05+00:00 Generating deployment configuration..
2024-10-14 20:09:06+00:00 Submitting deployment to compute..
2024-10-14 20:09:16+00:00 Checking the status of deployment anomaly-detector-service..
2024-10-14 20:10:25+00:00 Checking the status of inference endpoint anomaly-detector-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service deployed at: http://3e863a0d-d990-4a1b-926e-1d1ffe7a73fc.swedencentral.azurecontainer.io/score


# Enviando Dados para o Serviço de Inferência no Azure Machine Learning

Este código faz uma requisição **POST** para um serviço de inferência no **Azure Machine Learning**. Ele envia dados de exemplo ao modelo implantado e retorna as predições do modelo. Abaixo está a explicação de cada parte do código.

## 1. **Importando Bibliotecas Necessárias**

```python
import requests
import json
```

- **`requests`**: Biblioteca usada para enviar requisições HTTP, neste caso, uma requisição **POST** para o serviço de inferência no Azure.
- **`json`**: Biblioteca usada para converter dados de entrada para o formato JSON, que é o formato aceito pelo serviço de inferência.

## 2. **Definindo a URL do Serviço de Inferência**

```python
scoring_uri = service.scoring_uri
```

- A variável `scoring_uri` armazena a URL (URI de pontuação) do serviço de inferência implantado no Azure.
- **Nota**: Se você estiver executando o código localmente, precisaria substituir essa variável pela **URI real** do serviço de inferência, que pode ser algo como `"https://<nome-do-serviço>.azurewebsites.net/score"`.

## 3. **Definindo os Dados de Exemplo para Teste**

```python
input_data = {
    "input": [
        [0.5],  # Dados normais (previsão esperada: 1)
        [10.0],  # Dados fora do padrão (esperado: -1)
        [0.4]   # Dados normais (previsão esperada: 1)
    ]
}
```

- Os dados de entrada são fornecidos no formato de um dicionário Python. 
  - Cada lista interna (por exemplo, `[0.5]` ou `[10.0]`) representa uma amostra de dados que o modelo utilizará para fazer a previsão.
  - A previsão esperada seria:
    - **1** para dados normais (ex: `[0.5]` e `[0.4]`)
    - **-1** para dados fora do padrão (ex: `[10.0]`).

## 4. **Convertendo os Dados para JSON**

```python
input_data_json = json.dumps(input_data)
```

- A função `json.dumps()` converte o dicionário de dados Python em uma string no formato **JSON**. Isso é necessário porque as APIs de inferência geralmente esperam os dados no formato JSON.

## 5. **Cabeçalhos da Requisição**

```python
headers = {"Content-Type": "application/json"}
```

- O cabeçalho da requisição indica que os dados estão no formato **JSON** (`Content-Type: application/json`). Isso ajuda o serviço de inferência a interpretar corretamente os dados de entrada.

## 6. **Enviando a Requisição para o Serviço**

```python
response = requests.post(scoring_uri, data=input_data_json, headers=headers)
```

- A função `requests.post()` envia uma requisição **POST** para o serviço web de inferência no Azure.
  - **`scoring_uri`**: A URL do serviço de inferência, onde os dados serão enviados.
  - **`data=input_data_json`**: Os dados de entrada no formato JSON.
  - **`headers=headers`**: Os cabeçalhos da requisição que especificam que o conteúdo enviado é JSON.

## 7. **Exibindo a Resposta**

```python
print(response.json())
```

- A função `response.json()` converte a resposta da API de volta para um formato Python (geralmente um dicionário), que é impresso no console.
- A resposta do serviço provavelmente incluirá uma lista de predições para os dados fornecidos. Cada predição será:
  - **1** para amostras que o modelo considera normais.
  - **-1** para amostras que o modelo considera anômalas.

## Resumo

Este código realiza uma requisição POST para um serviço de inferência no **Azure Machine Learning**, enviando dados de entrada e recebendo predições do modelo implantado. Os dados de entrada são convertidos para JSON e enviados para a **URI de pontuação** do serviço, e a resposta, que contém as predições do modelo, é exibida no console. Esse fluxo é útil para testar o comportamento do modelo com diferentes entradas e garantir que o serviço esteja funcionando corretamente em produção.

In [34]:
import requests
import json

# URL do serviço implantado (substitua pela URI correta)
scoring_uri = service.scoring_uri

# Dados de exemplo para teste
input_data = {
    "input": [
        [0.5],  # Dados normais (previsão esperada: 1)
        [10.0],  # Dados fora do padrão (esperado: -1)
        [0.4]   # Dados normais (previsão esperada: 1)
    ]
}

# Converter dados para JSON
input_data_json = json.dumps(input_data)

# Cabeçalhos da requisição
headers = {"Content-Type": "application/json"}

# Enviar a requisição POST para o endpoint
response = requests.post(scoring_uri, data=input_data_json, headers=headers)

# Exibir a resposta
print(response.json())


{'predictions': [1, 1, 1], 'alerts': []}


# Recuperando logs de execução

A função `service.get_logs` é usada para recuperar os logs gerados pelo serviço de inferência implantado no **Azure Machine Learning**. Esses logs podem ser úteis para depurar o comportamento do serviço, monitorar erros ou verificar o estado da implantação.

No entanto, para realmente exibir os logs, você precisa chamar a função `get_logs()` com parênteses, assim:

```python
print(service.get_logs())
```

Essa linha de código:

- **`service.get_logs()`**: Chama o método que recupera os logs do serviço.
- **`print()`**: Exibe os logs no console.

## O que esperar dos logs

Os logs retornados podem incluir:
- Mensagens de inicialização do serviço.
- Logs do contêiner que executa o modelo.
- Logs de requisições HTTP recebidas, incluindo status de sucesso ou erro.
- Erros de inferência, se houver.

Esses logs são úteis para verificar se o serviço está funcionando corretamente ou para depurar possíveis problemas no código ou no ambiente de inferência.

In [10]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='ml-workspace-training', subscription_id='32855dba-b7b2-4fdb-8288-f0615afa0cdf', resource_group='rg-evaluations'), name=anomaly-detector-service, image_id=None, image_digest=None, compute_type=ACI, state=Unhealthy, scoring_uri=http://980ecd8c-495f-4619-9f44-dc05a0afcd4a.swedencentral.azurecontainer.io/score, tags={}, properties={'hasInferenceSchema': 'False', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': '387398dd-48ba-4eb8-9f5b-9d7b373ea1c0', 'userPuId': '10032002FD96959D', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/6bd72601-6bad-49d9-82df-a390b167cd68/', 'userTenantId': '6bd72601-6bad-49d9-82df-a390b167cd68', 'userName': 'Rodrigo Tavares', 'upn': 'tavares@netplataforma.com.br'})>
